In [1]:
import pandas as pd
import os
import numpy as np
import opendatasets as od 

In [4]:
user_data = pd.read_csv('steam-video-games\steam-200k.csv', encoding='ISO-8859-1', usecols=[0, 1, 2, 3], names=['UserID', 'Game', 'Behavior', 'Quantity'])
hours_played = user_data[user_data['Behavior'] == 'play'].groupby('Game').agg(np.mean).sort_values(by='Quantity', ascending=False).drop('UserID', axis='columns')
normalized_user_data = user_data.copy()[user_data['Game'].isin(hours_played.index)]
normalized_user_data['Quantity'] = normalized_user_data.apply(lambda x: x['Quantity'] if x['Behavior'] == 'purchase' else x['Quantity'] / hours_played.loc[x['Game'], :]['Quantity'], axis=1)
grouped_users = normalized_user_data.groupby(['UserID', 'Game']).aggregate({'Quantity': 'sum'})
grouped_users = grouped_users.reset_index(level='Game')
pivoted_users = grouped_users.reset_index()
pivoted_users = pivoted_users.set_index(['UserID', 'Game'])
pivoted_users = pivoted_users.unstack(fill_value=0)
pivoted_users.head(10)

Quantity            \
Game   007 Legends 0RBITALIS   
UserID                         
5250           0.0       0.0   
76767          0.0       0.0   
86540          0.0       0.0   
103360         0.0       0.0   
144736         0.0       0.0   
181212         0.0       0.0   
229911         0.0       0.0   
298950         0.0       0.0   
299153         0.0       0.0   
381543         0.0       0.0   

                                                                   \
Game   1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby)   
UserID                                                              
5250                                                  0.0           
76767                                                 0.0           
86540                                                 0.0           
103360                                                0.0           
144736                                                0.0           
181212                                                0.0           
229911                                                0.0           
298950                                                0.0           
299153                                                0.0           
381543                                                0.0           

                                                               \
Game   10 Second Ninja 10,000,000 100% Orange Juice 1000 Amps   
UserID                                                          
5250               0.0        0.0               0.0       0.0   
76767              0.0        0.0               0.0       0.0   
86540              0.0        0.0               0.0       0.0   
103360             0.0        0.0               0.0       0.0   
144736             0.0        0.0               0.0       0.0   
181212             0.0        0.0               0.0       0.0   
229911             0.0        0.0               0.0       0.0   
298950             0.0        0.0               0.0       0.0   
299153             0.0        0.0               0.0       0.0   
381543             0.0        0.0               0.0       0.0   

                                                                         \
Game   12 Labours of Hercules 12 Labours of Hercules II The Cretan Bull   
UserID                                                                    
5250                      0.0                                       0.0   
76767                     0.0                                       0.0   
86540                     0.0                                       0.0   
103360                    0.0                                       0.0   
144736                    0.0                                       0.0   
181212                    0.0                                       0.0   
229911                    0.0                                       0.0   
298950                    0.0                                       0.0   
299153                    0.0                                       0.0   
381543                    0.0                                       0.0   

                                              ...                             \
Game   12 Labours of Hercules III Girl Power  ... rFactor rFactor 2 realMyst   
UserID                                        ...                              
5250                                     0.0  ...     0.0       0.0      0.0   
76767                                    0.0  ...     0.0       0.0      0.0   
86540                                    0.0  ...     0.0       0.0      0.0   
103360                                   0.0  ...     0.0       0.0      0.0   
144736                                   0.0  ...     0.0       0.0      0.0   
181212                                   0.0  ...     0.0       0.0      0.0   
229911                                   0.0  ...     0.0       0.0      0.0   
298950                                   0.0  ...     0.0       0.0      

In [7]:
biggest_user = pivoted_users.sum(axis=1).idxmax()
biggest_user_index = pivoted_users.index.get_loc(biggest_user)
biggest_user

62990992

In [11]:
pivoted_users.loc[biggest_user].sort_values(ascending=False)[:10]

          Game                          
Quantity  Alien Swarm                       16.938206
          Relic Hunters Zero                10.446328
          Woodle Tree Adventures             9.384058
          Uncrowded                          7.369427
          The Hat Man Shadow Ward            6.672727
          Dungeon Defenders                  6.671873
          Oddworld Abe's Oddysee             6.176056
          Viking Battle for Asgard           6.154070
          Phantom Breaker Battle Grounds     5.847561
          Gravilon                           5.490323
Name: 62990992, dtype: float64

In [47]:
user_games = pivoted_users.loc[biggest_user].reset_index().set_index('Game').drop('level_0', axis='columns').rename(columns={biggest_user: 'Quantity'})
user_games = user_games[user_games['Quantity'] != 0.0]
user_games

,Quantity
Game,
1000 Amps,1.000000
12 Labours of Hercules,2.976945
12 Labours of Hercules II The Cretan Bull,3.309038
12 Labours of Hercules III Girl Power,2.821429
16bit Trader,2.056180
...,...
Zeno Clash,1.835821
Zombie Driver HD,3.461538
Zombie Zoeds,1.711111


In [12]:
if not os.path.exists('steam-video-games'):
    od.download( 
        "https://www.kaggle.com/datasets/tamber/steam-video-games/data") 

game_stats = pd.read_csv('steam-games-dataset/games.csv', encoding='ISO-8859-1', usecols = [0, 1, 2, 22, 23, 28, 35])
game_stats.head()

,AppID,Name,Release date,Positive,Negative,Average playtime forever,Genres
0,20200,Galactic Bowling,"Oct 21, 2008",6,11,0,"Casual,Indie,Sports"
1,655370,Train Bandit,"Oct 12, 2017",53,5,0,"Action,Indie"
2,1732930,Jolt Project,"Nov 17, 2021",0,0,0,"Action,Adventure,Indie,Strategy"
3,1355720,Henosisâ¢,"Jul 23, 2020",3,0,0,"Adventure,Casual,Indie"
4,1139950,Two Weeks in Painland,"Feb 3, 2020",50,8,0,"Adventure,Indie"


In [13]:
game_stats['Genres'] = game_stats['Genres'].apply(lambda row: str(row).split(','))
game_stats.head()

,AppID,Name,Release date,Positive,Negative,Average playtime forever,Genres
0,20200,Galactic Bowling,"Oct 21, 2008",6,11,0,"[Casual, Indie, Sports]"
1,655370,Train Bandit,"Oct 12, 2017",53,5,0,"[Action, Indie]"
2,1732930,Jolt Project,"Nov 17, 2021",0,0,0,"[Action, Adventure, Indie, Strategy]"
3,1355720,Henosisâ¢,"Jul 23, 2020",3,0,0,"[Adventure, Casual, Indie]"
4,1139950,Two Weeks in Painland,"Feb 3, 2020",50,8,0,"[Adventure, Indie]"


In [ ]:
list(pivoted_users['Quantity'].columns)

In [ ]:
game_stats['Name'].to_list()

In [43]:
overlapping_games = set(list(pivoted_users['Quantity'].columns)).intersection(set(game_stats['Name'].to_list()))

In [55]:
joined_games = user_games.merge(game_stats[['Name', 'Genres']], left_on='Game', right_on='Name').set_index('Name')
joined_games.head()

,Quantity,Genres
Name,,
1000 Amps,1.000000,"[Adventure, Indie]"
12 Labours of Hercules,2.976945,"[Casual, Strategy]"
16bit Trader,2.056180,"[Adventure, Casual, Indie, RPG, Simulation, St..."
1Quest,2.000000,"[Indie, RPG]"
8BitMMO,1.000000,"[Action, Adventure, Free to Play, Indie, Massi..."


In [65]:
genres = set(joined_games.Genres.explode().to_list())

{'Action',
 'Adventure',
 'Casual',
 'Design & Illustration',
 'Early Access',
 'Free to Play',
 'Gore',
 'Indie',
 'Massively Multiplayer',
 'RPG',
 'Racing',
 'Simulation',
 'Sports',
 'Strategy',
 'Violent',
 'Web Publishing',
 'nan'}

In [82]:
from collections import defaultdict

user_genre_dict = defaultdict(lambda: 0)
for game in joined_games.index:
    for genre in joined_games.loc[game]['Genres']:
        try:
            user_genre_dict[genre] += joined_games.loc[game]['Quantity'] / len(joined_games.loc[game]['Genres'])
        except:
            pass

# Normalize
sum_vals = 0
for val in user_genre_dict.values():
    sum_vals += val

for key in user_genre_dict:
    user_genre_dict[key] /= sum_vals

user_genre_dict

0.9999999999999999